In [1]:
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup
import regex
urllib3.disable_warnings()

In [2]:
from urllib.parse import quote 

The is a throwaway notebook that will later be converted into a script, once the parsing is more stable

In [3]:
universities = pd.read_csv('../data/Universities.csv', index_col=0)

We define universities that could be guessed based on the domain URL:

In [4]:
Full_uni = [i for i in universities.name.unique() if regex.search('[\p{Lu}]', i)]

In [5]:
location = []
n_found = []
for uni in Full_uni:
    #Only use the first part of university name, removing specifications
    u = regex.search('([\p{L}\ \'\-\.\&]*)', uni).group(0).strip()
    if ' - ' in u:
        u = regex.split(' - ', u)[0]
        
    #url with which we will query to find location
    url =u'https://en.wikipedia.org/wiki/'+quote(u)
    try:
        http_pool = urllib3.connection_from_url(url)
        r = http_pool.urlopen('GET',url)
        data = r.data
        soup = BeautifulSoup(data, 'html.parser')
        loc = soup.find_all('span', class_='geo-dec')[0].get_text()
        location.append((uni, loc))
        
    except:
        #list of universities for which we can not find a location
        n_found.append((uni, u))

In [6]:
pd.DataFrame(location, columns=['University', 'Location']).to_csv('../data/loclat.csv')

Number of universities we found:

In [7]:
len(location)

240

Number of Universities for which we couldn't find a match:

In [8]:
len(n_found)

22